In [ ]:
# --- Cell 1: Tải các thư viện ---
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import joblib
# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
print(f"XGBoost version: {xgb.__version__}")

In [ ]:
# --- Cell 2: Tải dữ liệu đã xử lý ---
file_name = 'kpi_processed.csv'
try:
    df = pd.read_csv(file_name, index_col=0, parse_dates=True)
    print(f"Tải file '{file_name}' thành công.")
    print(f"Dữ liệu có {len(df)} hàng, kéo dài từ {df.index.min()} đến {df.index.max()}.")
    df.info()
except Exception as e:
    print(f"LỖI: Không thể tải file '{file_name}'.")
    print(e)

In [ ]:
# --- Cell 3: Xử lý dữ liệu ---
if 'df' in locals():
    FEATURE_COLS = ['ps_traffic_mb', 'avg_rrc_connected_user', 'prb_dl_used', 'prb_dl_available_total', 'prb_utilization']
    N_FEATURES = len(FEATURE_COLS)
    print(f"Dự đoán {N_FEATURES} features: {FEATURE_COLS}")
    print("Đang xử lý: Đảm bảo dữ liệu 15 phút đầy đủ cho mỗi cell...")
    all_cells = df['cell_name'].unique()
    full_time_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='15min')
    multi_index = pd.MultiIndex.from_product([all_cells, full_time_index], names=['cell_name', 'timestamp'])
    df_data_only = df[FEATURE_COLS + ['cell_name']]
    df_data_only = df_data_only.reset_index()
    print(f"Dữ liệu trước khi xử lý trùng lặp: {len(df_data_only)} hàng")
    df_grouped_unique = df_data_only.groupby(['cell_name', 'timestamp']).mean()
    print(f"Dữ liệu sau khi xử lý trùng lặp (groupby.mean): {len(df_grouped_unique)} hàng")
    df_full = df_grouped_unique.reindex(multi_index, fill_value=0)
    df_full = df_full.reset_index()
    df_full = df_full.set_index('timestamp')
    print(f"Dữ liệu gốc có {len(df)} hàng.")
    print(f"Dữ liệu đã điền đầy đủ (sau reindex) có {len(df_full)} hàng.")
    print("Hoàn tất xử lý.")
    print("\n5 dòng đầu của dữ liệu đã xử lý đầy đủ:")
    print(df_full.head())
else:
    print("Lỗi: DataFrame 'df' không tồn tại. Vui lòng chạy lại Cell 2.")

In [ ]:
# --- Cell 4: Chia dữ liệu và Cấu hình mô hình ---
TIMESTEPS_PER_HOUR = 4
TIMESTEPS_PER_DAY = 24 * TIMESTEPS_PER_HOUR
INPUT_DAYS = 2
INPUT_STEPS = INPUT_DAYS * TIMESTEPS_PER_DAY
OUTPUT_DAYS = 1
OUTPUT_STEPS = OUTPUT_DAYS * TIMESTEPS_PER_DAY
print(f"Cấu hình mô hình: Input {INPUT_STEPS} bước (2 ngày), Output {OUTPUT_STEPS} bước (1 ngày).")

if 'df_full' in locals():
    total_duration = df_full.index.max() - df_full.index.min()
    train_end_time = df_full.index.min() + total_duration * 0.7
    val_end_time = df_full.index.min() + total_duration * 0.9
    
    train_df = df_full[df_full.index < train_end_time]
    val_df = df_full[(df_full.index >= train_end_time) & (df_full.index < val_end_time)]
    test_df = df_full[df_full.index >= val_end_time]
    
    print(f"Kích thước tập Train: {train_df.shape}")
    print(f"Kích thước tập Val:   {val_df.shape}")
    print(f"Kích thước tập Test:  {test_df.shape}")
else:
    print("Lỗi: DataFrame 'df_full' không tồn tại.")

In [ ]:
# --- Cell 5: Chuẩn hóa (Scaling) Dữ liệu ---
if 'train_df' in locals():
    train_df = train_df.replace([np.inf, -np.inf], 0).fillna(0)
    val_df = val_df.replace([np.inf, -np.inf], 0).fillna(0)
    test_df = test_df.replace([np.inf, -np.inf], 0).fillna(0)

    # Scaler đã được tạo và lưu từ notebook catboost, ở đây chỉ cần load lại
    try:
        scaler = joblib.load('scaler.gz')
        print("Load scaler thành công từ 'scaler.gz'.")
    except FileNotFoundError:
        print("Lỗi: File 'scaler.gz' không tìm thấy. Hãy chạy notebook catboost trước.")
        scaler = None

    if scaler:
        train_cells = train_df['cell_name']
        val_cells = val_df['cell_name']
        test_cells = test_df['cell_name']

        train_scaled_data = scaler.transform(train_df[FEATURE_COLS])
        val_scaled_data = scaler.transform(val_df[FEATURE_COLS])
        test_scaled_data = scaler.transform(test_df[FEATURE_COLS])

        scaled_train_df = pd.DataFrame(train_scaled_data, columns=FEATURE_COLS, index=train_df.index)
        scaled_train_df['cell_name'] = train_cells

        scaled_val_df = pd.DataFrame(val_scaled_data, columns=FEATURE_COLS, index=val_df.index)
        scaled_val_df['cell_name'] = val_cells

        scaled_test_df = pd.DataFrame(test_scaled_data, columns=FEATURE_COLS, index=test_df.index)
        scaled_test_df['cell_name'] = test_cells
        
        print("Hoàn tất scaling dữ liệu.")
else:
    print("Lỗi: DataFrame 'train_df' không tồn tại.")

In [ ]:
# --- Cell 6: Hàm tạo cửa sổ (Windowing) ---
def create_windows(data_df, input_steps, output_steps, feature_cols):
    X, y = [], []
    grouped = data_df.groupby('cell_name')
    total_cells = len(grouped)
    print(f"Bắt đầu tạo cửa sổ cho {total_cells} cell...")
    
    for cell_id, cell_data in grouped:
        cell_features = cell_data[feature_cols].values
        total_samples = len(cell_features)
        total_window_len = input_steps + output_steps
        
        for i in range(total_samples - total_window_len + 1):
            input_end = i + input_steps
            output_end = input_end + output_steps
            
            window_X = cell_features[i:input_end, :]
            window_y = cell_features[input_end:output_end, :]
            
            X.append(window_X)
            y.append(window_y)
            
    print(f"Hoàn tất tạo cửa sổ. Đã tạo {len(X)} mẫu.")
    return np.array(X), np.array(y)

In [ ]:
# --- Cell 7: Áp dụng hàm Windowing và làm phẳng dữ liệu ---
if 'scaled_train_df' in locals():
    print("--- Đang tạo mẫu Train ---")
    X_train, y_train = create_windows(scaled_train_df, INPUT_STEPS, OUTPUT_STEPS, FEATURE_COLS)
    print("\n--- Đang tạo mẫu Validation ---")
    X_val, y_val = create_windows(scaled_val_df, INPUT_STEPS, OUTPUT_STEPS, FEATURE_COLS)
    print("\n--- Đang tạo mẫu Test ---")
    X_test, y_test = create_windows(scaled_test_df, INPUT_STEPS, OUTPUT_STEPS, FEATURE_COLS)

    # Làm phẳng dữ liệu (Flatten)
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    y_train_flat = y_train.reshape(y_train.shape[0], -1)
    X_val_flat = X_val.reshape(X_val.shape[0], -1)
    y_val_flat = y_val.reshape(y_val.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)
    # y_test sẽ được giữ nguyên dạng 3D để so sánh

    print("\n--- Kích thước dữ liệu sau khi làm phẳng ---")
    print(f"X_train_flat shape: {X_train_flat.shape}")
    print(f"y_train_flat shape: {y_train_flat.shape}")
    print(f"X_val_flat shape:   {X_val_flat.shape}")
    print(f"y_val_flat shape:   {y_val_flat.shape}")
    print(f"X_test_flat shape:  {X_test_flat.shape}")
    print(f"y_test shape:       {y_test.shape}")
else:
    print("Lỗi: Dữ liệu đã scale không tồn tại.")

In [ ]:
# --- Cell 8: Xây dựng và Huấn luyện mô hình XGBoost ---
from sklearn.multioutput import MultiOutputRegressor

if 'X_train_flat' in locals():
    print("Bắt đầu huấn luyện XGBoost...")
    
    # XGBoost cũng không hỗ trợ trực tiếp multi-output trong kịch bản này
    # Chúng ta lại dùng MultiOutputRegressor
    xgb_model = xgb.XGBRegressor(
        n_estimators=1000,      # Tương tự iterations trong CatBoost
        learning_rate=0.1,
        max_depth=6,
        objective='reg:squarederror', # Tương đương MSE
        random_state=42,
        n_jobs=-1,              # Sử dụng tất cả các CPU cores
        eval_metric='rmse'      # Metric để theo dõi
    )
    
    # Bao bọc với MultiOutputRegressor
    multi_output_model_xgb = MultiOutputRegressor(xgb_model)

    # Huấn luyện
    # Để sử dụng early stopping với XGBoost trong MultiOutputRegressor,
    # chúng ta cần một chút tùy chỉnh hoặc chấp nhận không dùng nó.
    # Ở đây, để đơn giản, chúng ta sẽ không dùng early stopping.
    multi_output_model_xgb.fit(X_train_flat, y_train_flat)

    print("Hoàn tất huấn luyện!")
    
    # Lưu mô hình
    # Lưu ý: MultiOutputRegressor không có hàm save_model trực tiếp.
    # Chúng ta lưu thông qua joblib.
    joblib.dump(multi_output_model_xgb, 'xgboost_multi_model.joblib')
    print("Đã lưu mô hình vào 'xgboost_multi_model.joblib'")

    # Để lấy loss history, chúng ta cần truy cập vào từng estimator con.
    # Tuy nhiên, do cách MultiOutputRegressor hoạt động, nó không truyền
    # eval_set vào các model con một cách trực tiếp khi fit. 
    # Việc lấy history sẽ phức tạp hơn và cần vòng lặp huấn luyện thủ công.
    # Trong khuôn khổ bài này, chúng ta sẽ bỏ qua việc vẽ biểu đồ loss cho XGBoost.

else:
    print("Lỗi: Thiếu dữ liệu đã xử lý (X_train_flat).")

In [ ]:
# --- Cell 9: Đánh giá trên tập Test ---
if 'multi_output_model_xgb' in locals():
    print("--- Đánh giá XGBoost trên tập Test ---")
    
    y_pred_flat = multi_output_model_xgb.predict(X_test_flat)
    y_test_flat = y_test.reshape(y_test.shape[0], -1)

    test_loss_mse = mean_squared_error(y_test_flat, y_pred_flat)
    test_metric_mae = mean_absolute_error(y_test_flat, y_pred_flat)

    print(f"  Test Loss (MSE): {test_loss_mse:.6f}")
    print(f"  Test MAE (Mean Absolute Error): {test_metric_mae:.6f}")
    
    with open('xgboost_loss.txt', 'w') as f:
        f.write(f"MSE: {test_loss_mse}\n")
        f.write(f"MAE: {test_metric_mae}\n")
    print("Đã lưu loss vào file 'xgboost_loss.txt'.")
else:
    print("Lỗi: Mô hình XGBoost chưa được huấn luyện.")

In [ ]:
# --- Cell 10: So sánh kết quả dự đoán với giá trị thực tế ---
if 'multi_output_model_xgb' in locals() and 'X_test' in locals() and 'scaler' in locals():
    print("--- Trực quan hóa kết quả dự đoán XGBoost trên tập Test ---")
    num_samples_to_plot = min(5, X_test.shape[0])
    if num_samples_to_plot > 0:
        y_pred_flat = multi_output_model_xgb.predict(X_test_flat[:num_samples_to_plot])
        y_pred_scaled = y_pred_flat.reshape(num_samples_to_plot, OUTPUT_STEPS, N_FEATURES)

        y_test_subset = y_test[:num_samples_to_plot]

        y_true_reshaped = y_test_subset.reshape(-1, N_FEATURES)
        y_true_original = scaler.inverse_transform(y_true_reshaped)
        y_true_original = y_true_original.reshape(num_samples_to_plot, OUTPUT_STEPS, N_FEATURES)

        y_pred_reshaped = y_pred_scaled.reshape(-1, N_FEATURES)
        y_pred_original = scaler.inverse_transform(y_pred_reshaped)
        y_pred_original = y_pred_original.reshape(num_samples_to_plot, OUTPUT_STEPS, N_FEATURES)

        for i in range(num_samples_to_plot):
            print(f"\n--- So sánh Mẫu #{i+1} ---")
            fig, axes = plt.subplots(N_FEATURES, 1, figsize=(15, 5 * N_FEATURES), sharex=True)
            if N_FEATURES == 1: axes = [axes]

            for j, feature_name in enumerate(FEATURE_COLS):
                ax = axes[j]
                ax.plot(y_true_original[i, :, j], label=f'Thực tế ({feature_name})', color='blue', marker='o', linestyle='-')
                ax.plot(y_pred_original[i, :, j], label=f'Dự đoán ({feature_name})', color='red', marker='x', linestyle='--')
                ax.set_title(f'Dự đoán {feature_name} (Mẫu #{i+1})')
                ax.set_xlabel('Bước thời gian (15 phút)')
                ax.set_ylabel(f'Giá trị {feature_name}')
                ax.legend()
                ax.grid(True)
            
            plt.tight_layout()
            plt.show()
    else:
        print("Không có mẫu nào trong X_test để dự đoán.")
else:
    print("Lỗi: Không thể trực quan hóa.")